In [ ]:
from typing import List, Dict
import html
import os
from sklearn.metrics import  mean_absolute_percentage_error
import re
import numpy as np
from tqdm import tqdm
from scipy.optimize import curve_fit
import scipy
import matplotlib.pyplot as plt

In [ ]:
# Список точек на сколько надо сдвинуть вектора между собой чтобы посчитать значение корреляции 
POWER = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600, 700, 800, 900,
         1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 20000, 30000, 40000]

In [ ]:
def preprocess_text(text):
    """
    Convert all named and numeric character references (e.g. &gt;, &#62;,
    &x3e;) in the string s to the corresponding unicode characters.
    This function uses the rules defined by the HTML 5 standard
    for both valid and invalid character references, and the list of
    HTML 5 named character references defined in html.entities.html5.
    Args:
        text: Book text.

    Returns:
        Book text in lowercase letters.
    """
    text = html.unescape(text)  # &amp;#x200B; => &#x200B;
    text = html.unescape(text)  # &amp;#x200B; =>
    return text.lower()


def tokenize(text):
    """
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the Match object and must return
    a replacement string to be used.
    Args:
        text: Book text.

    Returns:
        List of words.
    """
    opt = re.sub(r'[^\w\s]', '', text)
    words = opt.split()
    return words

In [ ]:
def get_embeddings(path_glove: str) -> Dict[str, np.ndarray]:
    """
    The function returns a dictionary, the key is a word, the value is a vector embedding
    Args:
        path_glove: Path to the glove file.

    Returns:
        The function returns a dictionary Dict[str, np.ndarray], the key is a word, the value is a vector embedding
    """
    embeddings = {}
    with open(path_glove, 'r', encoding='utf-8') as file:

        for line in tqdm(file):
            values = line.split()
            word = values[0]
            embedding = np.asarray(values[1:], dtype='float32')
            embeddings[word] = embedding
    return embeddings


def get_tokens(path_text: str) -> List[str]:
    """
    The function returns a list of tokens from text.
    Args:
        path_text: Path to the text file.

    Returns:
        List of tokens from text.
    """
    tokens = []
    with open(path_text, encoding='utf-8') as file:
        for line in file:
            tokens += tokenize(preprocess_text(line))
    return tokens

In [ ]:
def get_tokens_with_embeddings(embeddings: np.ndarray, tokens: List, verbose: bool = False) -> np.ndarray:
    """
    The function finds embeddings of words that are in the book.
    Args:
        embeddings: Embeddings.
        tokens: Tokens.
        verbose: Console output of a word that is not in the dictionary glove
        (False do not output to console, True output to console).

    Returns:
        Embeddings of words that are in the book.
    """
    tokens_with_embeddings = []
    for token in tokens:
        try:
            tokens_with_embeddings.append(embeddings[token])
        except KeyError:
            if verbose:
                print('Not found', token)

    return np.asarray(tokens_with_embeddings, dtype='float32')


def get_normalized_embeddings(embeddings: np.ndarray) -> np.ndarray:
    """
    The function finds the weighted average values of embeddings.
    Args:
        embeddings: Embeddings.

    Returns:
        The function returns array weighted average values of embeddings.
    """
    quantity, dim = embeddings.shape
    avg = np.zeros(dim)
    for i in range(quantity):
        avg += embeddings[i]
    avg = avg / quantity

    for i in range(quantity):
        embeddings[i] -= avg

    return embeddings


def get_norms_embeddings(embeddings: np.ndarray) -> np.ndarray:
    """
    The function finds the norm of vectors.
    Args:
        embeddings: Embeddings.

    Returns:
        Norm of word vectors.
    """
    quantity = len(embeddings)
    norms = np.zeros(quantity)

    for i in range(quantity):
        norms[i] = np.linalg.norm(embeddings[i])
    return norms

In [ ]:
def get_autocorrelation(embeddings: np.ndarray, norms: np.ndarray, number_embeddings: int) -> np.ndarray:
    """
    The function finds the autocorrelation values of word vectors
    Args:
        embeddings: Embeddings.
        norms: Norm of word vectors.
        number_embeddings: Number of Embeddings.

    Returns:
        Array of correlation values
    """
    autocorr = np.zeros(len(POWER))
    for k in tqdm(range(len(POWER))):
        sumcorr = 0.
        j = POWER[k]
        for i in range(number_embeddings - j):
            corr = np.dot(embeddings[i], embeddings[i + j]) / norms[i] / norms[i + j]
            sumcorr += corr
        autocorr[k] = sumcorr / (number_embeddings - j)
    return autocorr

In [ ]:
def get_negative_index(values_y):
    """The method finds the extreme (maximum) index of a non-negative value"""
    neg = np.where(values_y <= 0)[0]
    if neg.size > 0:
        return np.max(neg) + 1
    else:
        return 0
    
    
def mapping(x: np.array, a: float, b: float) -> np.array:
    """
    The method approximates the direct correlation value.

    Args:
        x: the coordinates of any point on the line
        a: is the slope of the line.
        b: is the y-intercept (where the line crosses the y-axis).

    Return:
        Returns an array of numbers.
    """
    return a * x + b


def get_power_mape(values_x, values_y, negative_index):
    """
        The method finds the average absolute percentage error for a power law.
    Returns:
        The method returns the average absolute percentage error for a power law.
    """
    argspow, _ = curve_fit(mapping, np.log10(values_x[negative_index:]), np.log10(values_y[negative_index:]))
    y_fitpow = mapping(np.log10(values_x[negative_index:]), argspow[0], argspow[1])
    return argspow, y_fitpow, mean_absolute_percentage_error(values_y[negative_index:], 10 ** y_fitpow)


def get_log_mape(values_x, values_y, negative_index):
    """
        The method finds the average absolute percentage error for a logarithmic law.
    Returns:
        The method returns the average absolute percentage error for a logarithmic law.
    """
    argslog, _ = curve_fit(mapping, np.log10(values_x[negative_index:]), values_y[negative_index:])
    y_fitlog = mapping(np.log10(values_x[negative_index:]), argslog[0], argslog[1])
    return argslog, y_fitlog, mean_absolute_percentage_error(values_y[negative_index:], y_fitlog)


def get_exp_mape(values_x, values_y, negative_index):
    """
        The method finds the average absolute percentage error for an exponential law.
    Returns:
        The method returns the average absolute percentage error for an exponential law.
    """
    argsexp, _ = curve_fit(mapping, values_x[negative_index:], np.log10(values_y[negative_index:]))
    y_fitexp = mapping(values_x[negative_index:], argsexp[0], argsexp[1])
    return argsexp, y_fitexp, mean_absolute_percentage_error(values_y[negative_index:], 10 ** y_fitexp)

In [ ]:

# Заменить пути на свои!!!
path_glove = 'Glove/multilingual_embeddings.en'
path_text = 'LongText/Don Quijote de la Mancha EN.txt'  
    
_, filename = os.path.split(path_text)

embeddings = get_embeddings(path_glove)
print('Found %s word vectors.' % len(embeddings))
tokens = get_tokens(path_text)
print('All tokens in', filename, len(tokens))
tokens_with_embeddings = get_tokens_with_embeddings(embeddings, tokens, verbose=False)
print("tokens with embeddings", len(tokens_with_embeddings))

normalized_embeddings = get_normalized_embeddings(tokens_with_embeddings)
print('Embeddings normalized', normalized_embeddings.shape, normalized_embeddings.mean())

quantity_embeddings = len(normalized_embeddings)
norms = get_norms_embeddings(normalized_embeddings)
autocorr = get_autocorrelation(normalized_embeddings, norms, quantity_embeddings)
print(filename)


negative_index = get_negative_index(autocorr)
print('Power decay')
argspow, y_fitpow, MAPE_POWER = get_power_mape(np.array(POWER), autocorr, negative_index)
print(f'a = {argspow[0]},  b = {argspow[1]}')
print(f'Корреляция Пирсона =', scipy.stats.pearsonr(np.log10(autocorr[negative_index:]), y_fitpow))
print('MAPE=', MAPE_POWER)
print()
print('Log decay')
argslog, y_fitlog, MAPE_LOG = get_log_mape(np.array(POWER), autocorr, negative_index)
print(f'a = {argslog[0]},  b = {argslog[1]}')
print(f'Корреляция Пирсона =', scipy.stats.pearsonr(np.log10(autocorr[negative_index:]), y_fitlog))
print('MAPE=', MAPE_LOG)
print()
print('Exp decay')
argsexp, y_fitexp, MAPE_EXP = get_exp_mape(np.array(POWER), autocorr, negative_index)
print(f'a = {argsexp[0]},  b = {argsexp[1]}')
print(f'Корреляция Пирсона =', scipy.stats.pearsonr(np.log10(autocorr[negative_index:]), y_fitexp))
print('MAPE=', MAPE_EXP)
print()
print('GAPELMAPER')
print(MAPE_POWER / MAPE_EXP)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 15))

# add data to plots
ax[0, 0].plot(POWER, autocorr, label='corr')
ax[0, 0].grid(True)
ax[0, 0].set_xlabel('Log distance', size='large')
ax[0, 0].set_ylabel('Log Correlation', size='large')
ax[0, 0].set_yscale('log')
ax[0, 0].set_xscale('log')

ax[0, 1].plot(POWER, autocorr, label='corr')
ax[0, 1].grid(True)
ax[0, 1].set_xlabel('Distance', size='large')
ax[0, 1].set_ylabel('Log Correlation', size='large')
ax[0, 1].set_yscale('log')

ax[1, 0].plot(POWER, autocorr, label='corr')
ax[1, 0].grid(True)
ax[1, 0].set_xlabel('Log distance', size='large')
ax[1, 0].set_ylabel('Correlation', size='large')
ax[1, 0].set_xscale('log')

ax[1, 1].plot(POWER, autocorr, label='corr')
ax[1, 1].grid(True)
ax[1, 1].set_xlabel('Distance', size='large')
ax[1, 1].set_ylabel('Correlation', size='large')
plt.show()

In [ ]:
def plotting_the_graph_correlation(x: np.array, y_original: np.array, y_predict: np.array, logx: str, logy: str):
    """
    The function displays a graph of calculated correlation values,
    and a direct approximation of the correlation values.

    Args:
        x: Array of shifts of the matrix of attention vectors
        y_original: Array of correlation values
        y_predict: An array of values approximating the correlation
        logx: 
        logy:
    """
    plt.figure(figsize=(8, 6))
    plt.plot(x, y_original, 'bo', label="y - original")
    plt.plot(x, y_predict, label="y = a * x + b")
    plt.xlabel(logx + r'$\tau$', fontsize=24, fontweight="bold")  # ('𝜏')
    plt.ylabel(logy + r' C($\tau$)', fontsize=24, fontweight="bold")
    plt.legend(loc='best', fancybox=True, shadow=True)
    plt.grid(True)
    plt.show()

In [ ]:
values_x = np.log10(np.array(POWER)[negative_index:])
values_y = np.log10(autocorr[negative_index:])

plotting_the_graph_correlation(values_x, values_y, y_fitpow, 'Log',  'Log')

In [ ]:
values_x = np.log10(np.array(POWER)[negative_index:])
values_y = autocorr[negative_index:]

plotting_the_graph_correlation(values_x, values_y, y_fitlog, 'Log',  '')

In [ ]:
values_x = np.array(POWER)[negative_index:]
values_y = np.log10(autocorr[negative_index:])

plotting_the_graph_correlation(values_x, values_y, y_fitexp, '',  'Log')

In [ ]:
def get_discrepancy_plot(values_x: np.ndarray,
                         values_y: np.ndarray,
                         y_fitpow: np.ndarray,
                         y_fitlog: np.ndarray,
                         y_fitexp: np.ndarray,
                         negative_index: int = 0):
    discrep_power = values_y[negative_index:] - 10 ** y_fitpow
    discrep_log = values_y[negative_index:] - y_fitlog
    discrep_exp = values_y[negative_index:] - 10 ** y_fitexp

    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(14, 4))
    ax[0].plot(values_x[negative_index:], discrep_power, 'bo', label='discrepancy of power fit')
    ax[0].grid(True)
    ax[0].set_xlabel(r'$\tau$', fontsize=24, fontweight="bold")
    ax[0].set_ylabel(r'discrepancy', fontsize=12, fontweight="bold")
    ax[0].set_xscale('log')

    ax[1].plot(values_x[negative_index:], discrep_exp, 'bo', label='discrepancy of exp fit')
    ax[1].grid(True)
    ax[1].set_xlabel(r'$\tau$', fontsize=24, fontweight="bold")
    ax[1].set_ylabel(r'discrepancy', fontsize=12, fontweight="bold")
    ax[1].set_xscale('log')

    ax[2].plot(values_x[negative_index:], discrep_log, 'bo', label='discrepancy of log fit')
    ax[2].grid(True)
    ax[2].set_xlabel(r'$\tau$', fontsize=24, fontweight="bold")
    ax[2].set_ylabel(r'discrepancy', fontsize=12, fontweight="bold")
    ax[2].set_xscale('log')
    plt.show()
    

values_x = np.array(POWER)
values_y = autocorr

get_discrepancy_plot(values_x, values_y, y_fitpow, y_fitlog, y_fitexp, negative_index)